In [ ]:
!pip install selenium

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time

In [47]:
# webdirver 설정, 빅카인즈 사이트 이동

driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://www.bigkinds.or.kr/v2/news/index.do')

In [48]:
# 검색기간 1주일로 설정

driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="srch-tab1"]/div/div[1]/span[2]').click()

In [49]:
# 검색창의 이름을 찾아서 사용자가 검색어 입력

# query_txt = input('크롤링할 키워드는 무엇입니까?:')
searchbox = driver.find_element_by_id("total-search-key")
searchbox.send_keys("오리온")  
searchbox.send_keys("\n")  # 검색버튼 엔터

In [52]:
# 기사 분류 - 경제로 설정
# *주의* 화면을 크게해야 함. 경제 분류가 화면에 보여야 셀레니움이 잡을 수 있음

time.sleep(1)
driver.find_element_by_xpath('//*[@id="filterTab03"]/li[2]/span').click()

# from selenium.webdriver.common.keys import Keys
# .send_keys(Keys.ENTER)

In [53]:
# 기사 100건 씩 출력

# driver.find_element_by_xpath('//*[@id="select2"]/option[4]').click()

In [54]:
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

In [55]:
# 전체 페이지 개수

pageNum = soup.select('div.lastNum')
totalPage = int(pageNum[0]['data-page'])
totalPage

2

In [56]:
contents = []

# 현재 페이지
curPage = 1

while curPage <= totalPage:
    # bs4 초기화
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # 기사 리스트
    articles = soup.select('div.news-inner')
    # 페이지 번호 출력
    print('----- Current Page : {}'.format(curPage), '------')

    # 세부 데이터
    for article in articles:
        title = article.select_one('span.title-elipsis').text.strip()
        press =  article.select('div.info > div > a')
        category = article.select('div.info > div > span.bullet-keyword')
        date = article.select('div.info > p')
        contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date[1].text.strip()])
    
    # 페이지 수 증가
    curPage += 1
    if curPage > totalPage:
        print('Crawling succeed')
        break
        
    # 페이지 이동 클릭
    driver.implicitly_wait(3)      
    nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
    driver.execute_script("arguments[0].click();", nextbtn)
    
    # bs4 인스턴스 삭제
    del soup
    time.sleep(1.5)
 
# 브라우저 종료
driver.close() 

len(contents)

----- Current Page : 1 ------
----- Current Page : 2 ------
Crawling succeed!


17

In [57]:
contents


["[신상품라운지]오리온, '오리온 고소한 다과상' 추석 선물세트 출시",
 '경남제약 "레모나, 한-중 소비자가 선택 ‘2021 올해의 브랜드 대상’"',
 '오리온, 추석 맞이 한정판 ‘오리온 고소한 다과상’ 선물세트 출시',
 '본격 환절기…체력 보충 위한 건강 식음료 수요↑',
 "식품업계, 가을 환절기 앞두고 간편 영양 식음료 '주목'",
 '보릿고개 넘은 식품株…가을엔 볕들까',
 '삼다수 위탁판매 입찰전 뜨겁다...제3의 후보 등장',
 '비빔면소스 뿌린 피자 “물건이네”',
 '"삼성전자·SK하이닉스, 악재 다 반영…내년 하반기 D램 수요 살아난다" 박세익 전무 인터뷰',
 '식품업계, 가을 환절기 앞두고 간편 영양 식음료 인기 ‘쑥’',
 '충북, 전국품질분임조 경진대회 종합 3위',
 '박세익 전무 "재개방 수혜 소비재에서 스타 종목 나올 것"',
 '오리온, ‘산업계 전국체전’에서 2년 연속 대통령상',
 '오리온, 전국품질분임조경진대회 2년 연속 대통령상 금상',
 "오리온, '전국품질분임조경진대회' 2년 연속 대통령상 금상 수상",
 '인쇄용 동판 제조서 시작… 전자소재 MLCC 인쇄 롤 개발',
 '초콜릿으로 단백질 섭취 ‘건강한 단맛’ 인기']

In [22]:
contents = [] 

for i in range(pageNum):
    # 전체 기사들
    articles = driver.find_elements_by_css_selector('div.news-inner')

    for article in articles:
        title = article.find_elements_by_xpath('//*[@id="news-results"]/div[1]/div/div[2]/a/div/strong/span').text
        press =  article.find_elements_by_css_selector('div.info > div > a').text
        category = article.find_elements_by_css_selector('div.info > div > span.bullet-keyword').text
        date = article.find_elements_by_css_selector('div.info > p').text
        contents.append([title, press, category, date])
   
    try:
        driver.implicitly_wait(3) # seconds
        nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
        driver.execute_script("arguments[0].click();", nextbtn)
        time.sleep(1.5)
    except:
        print("데이터 수집 완료.")
        break
    
len(contents)


AttributeError: 'list' object has no attribute 'text'

+ 최근 1주일
1. 빅카인즈 경제 기사 키워드당 100개만 기져오기
2. 매일경제 + 한국경제 기사 가져오기

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 다음 페이지 넘기기

nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
browser.execute_script("arguments[0].click();", nextbtn)

# browser.find_element_by_css_selector('#news-results-tab > div.data-result-btm.m-only.paging-v3-wrp > div.btm-pav-wrp > div > div > div > div:nth-child(7) > a')
# nextbtn.send_keys(Keys.ENTER)

In [ ]:
contents = [] 

for i in range(99):
    time.sleep(3)
    articles = soup.select('div.news-inner')

    for article in articles:
        title = article.select('span.title-elipsis')
        press =  article.select('div.info > div > a')
        category = article.select('div.info > div > span.bullet-keyword')
        date = article.select('div.info > p')
        contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date[1].text.strip()])
   
    try:
        nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
        driver.execute_script("arguments[0].click();", nextbtn)
    except:
        print("데이터 수집 완료.")
        break
    
len(contents)


In [ ]:
print(contents)

In [ ]:
# [제목, 언론사, 분류, 날짜, 기자이름, 기사내용]

contents = [] 
for article in articles:
    title = article.select('span.title-elipsis')
    press =  article.select('div.info > div > a')
    category = article.select('div.info > div > span.bullet-keyword')
    date = article.select('div.info > p')
    contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date[1].text.strip()])

len(contents)

In [ ]:
contents[1]

In [ ]:
# 기사 전문 

# texts = []
# for i in range(0, 100):
#     driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
#     time.sleep(1)
#     text = driver.find_elements_by_css_selector('div.news-view-body')[0].text
#     texts.append(text.replace('\n', ''))
#     driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

# len(texts)

In [ ]:
df = pd.DataFrame(data=contents, columns =['title', 'press', 'category', 'date', 'name'])